In [1]:
import Pkg
Pkg.add("CSV")
Pkg.add("LinearAlgebra")
Pkg.add("Statistics")
Pkg.add("Plots")
Pkg.add("DataFrames")
using CSV
using Plots
using Statistics
using LinearAlgebra
using DataFrames

  Updating registry at `/srv/julia/pkg/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/Project.toml`
 [no changes]
  Updating `~/Manifest.toml`
 [no changes]


┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1273


In [2]:
dataset = CSV.read("housingPriceData.csv");


In [3]:
bedrooms=dataset.bedrooms;
bathrooms=dataset.bathrooms;
sqft_living=dataset.sqft_living;
Y=dataset.price;

function scale_features(s)

   μ = mean(s, dims=1);

   σ = std(s, dims=1);

 

   s_norm = (s .- μ) ./ σ;

 

    return (s_norm)

end
bedrooms=scale_features(bedrooms);
bathrooms=scale_features(bathrooms);
sqft_living=scale_features(sqft_living);


In [4]:
trainset = round(0.6*length(bedrooms));
crossvalidationset = round(0.2*length(bedrooms));
bedrooms_training = bedrooms[1:Int(trainset)];
bedrooms_validation = bedrooms[1+Int(trainset):1+Int(trainset)+Int(crossvalidationset)];
bedrooms_test = bedrooms[2+Int(trainset)+Int(crossvalidationset):length(bedrooms)];
bathrooms_training = bathrooms[1:Int(trainset)];
bathrooms_validation = bathrooms[1+Int(trainset):1+Int(trainset)+Int(crossvalidationset)];
bathrooms_test = bathrooms[2+Int(trainset)+Int(crossvalidationset):length(bedrooms)];
sqft_living_training = sqft_living[1:Int(trainset)];
sqft_living_validation = sqft_living[1+Int(trainset):1+Int(trainset)+Int(crossvalidationset)];
sqft_living_test = sqft_living[2+Int(trainset)+Int(crossvalidationset):length(bedrooms)];
Y_training = Y[1:Int(trainset)];
Y_validation = Y[1+Int(trainset):1+Int(trainset)+Int(crossvalidationset)];
Y_test = Y[2+Int(trainset)+Int(crossvalidationset):length(bedrooms)];


In [5]:
m_total = length(bedrooms);
x0_total = ones(m_total);
X_total = cat(x0_total,bedrooms,bathrooms,sqft_living,dims=2);
m_training = length(bedrooms_training);
x0_training = ones(m_training);
X_training = cat(x0_training, bedrooms_training, bathrooms_training, sqft_living_training, dims=2);
m_validation = length(bedrooms_validation);
x0_validation = ones(m_validation);
X_validation = cat(x0_validation, bedrooms_validation, bathrooms_validation, sqft_living_validation, dims=2);
m_test = length(bedrooms_test);
x0_test = ones(m_test);
X_test = cat(x0_test, bedrooms_test, bathrooms_test, sqft_living_test, dims=2);

In [22]:
function costFunctionRidge(X,Y,Theta,lambda)
    m=length(Y)
    #thetan=Theta
    #thetan[1]=0
    cost = (1/(2*m))*sum(((X*Theta)-Y).^2)+(lambda/(2*m))*sum((Theta).^2)
    
    return cost
end

costFunctionRidge (generic function with 1 method)

In [23]:
Theta = zeros(4,1);


In [26]:
function gradientDescentRidge(X,Y,Theta,lr,numIteration,lambda)
    m=length(Y)
    costHistory = zeros(numIteration);
    #thetan=Theta
    #thetan[1]=0
    
    for iteration in 1:numIteration
        loss = (X*Theta) - Y;
        gradient = (X' * loss)/m + (lambda/m)*Theta ;
        
        Theta = Theta - lr*gradient;
        cost = costFunctionRidge(X,Y,Theta,lambda);
        costHistory[iteration] = cost;
    end
   

    return Theta , costHistory 
end

gradientDescentRidge (generic function with 1 method)

In [45]:
lr =0.01
lambda = 100
newTheta ,costHistory = gradientDescentRidge(X_training,Y_training,Theta,lr,1000,lambda)

([536837.489600732; -51557.47592921969; 16863.156855259294; 272483.2784004962], [2.056830267694604e11, 2.0205084632836423e11, 1.9850588448580328e11, 1.9504572583156152e11, 1.9166803284834723e11, 1.8837054305873056e11, 1.8515106628578076e11, 1.8200748202265204e11, 1.789377369065719e11, 1.7593984229287906e11  …  3.5163957865437256e10, 3.516376940827953e10, 3.516358187177768e10, 3.516339525133594e10, 3.516320954238245e10, 3.516302474036915e10, 3.5162840840771576e10, 3.516265783908879e10, 3.5162475730843216e10, 3.51622945115805e10])

In [46]:
function cal(X,Y,Theta)
    m = length(Y)
    Ybar=mean(Y)*ones(m)
    RMSE=sqrt(sum((1/m)*((X*Theta-Y).^2)))
    R2=1-(sum((X*Theta-Y).^2)/sum((Y-Ybar).^2))    
    return RMSE,R2
end
RMSE ,R2 = cal(X_test,Y_test,newTheta)

(258431.80587440537, 0.5139010787381328)

In [47]:
Y_Pred_training = X_training*newTheta;
Y_Pred_validation= X_validation*newTheta;
Y_Pred_test = X_test*newTheta;
Y_Pred = X_total*newTheta;

In [50]:
df = DataFrame(Y_Pred)
CSV.write("2a.csv", df)

"2a.csv"